In [ ]:
%%javascript
$('#header').toggle();

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap, Circle
from ipywidgets import HTML, Layout, Dropdown, Text, Button, VBox, Box, Label 
from ipywidgets import AppLayout, GridspecLayout, interact
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy as np
import pandas as pd
from datetime import datetime

countries_file = 'sql-pays.csv'

geolocator = Nominatim(user_agent="appMAP_cernicchiaro.giulia@gmail.com")

maps = {'Standard' : basemaps.OpenStreetMap.Mapnik,
        'Esri' : basemaps.Esri.DeLorme}

state = sorted(list(pd.read_csv(countries_file, sep=',', low_memory=False, header=None)[4]))

#deps = {x['NOM']:x['DEP'] for i,x in pd.read_csv('C:\\deconMAP\\DepFra.csv', sep=',', low_memory=False).iterrows()}


header = HTML("<h2>Find Destinations Around Me</h2>", layout=Layout(height='auto'))
header.style.text_align='center'

basemap_selector = Dropdown( options = list(maps.keys()), layout=Layout(width='auto') )
basemap_selector.value = 'Standard'

state_selector = Dropdown( options = list(state), layout=Layout(width='auto') )
state_selector.value = 'France'

main_map = Map(basemap=basemaps.OpenStreetMap.Mapnik)
main_map.layout.height = 'auto'
center_map = geolocator.geocode(state_selector.value)
main_map.center = (center_map.latitude, center_map.longitude)
main_map.zoom = 5

circle = Circle()
circle_radius = Circle()

user_selection = Text(placeholder='Saisir une adresse ou cliquer sur la carte')
user_selection.layout.width = '80%'
#user_selection.layout.height = 'auto'

user_validation = Button(description='Valider', button_style='info', tooltip='Calculer le rayon')

#department_selector = Dropdown( options = list(deps.keys()), layout=Layout(width='auto') )
#department_selector.value = None

out = HTML(
    value='',
    layout=Layout(width='auto', height='auto')
)


rows = []

def add_log(msg):
    max_rows = 5
    rows.append(msg)
    if len(rows) > max_rows:
        rows.pop(0)
    out_rows=rows[:]
    out_rows.reverse()
    return '<h4>Activity log</h4><ul>{}</ul>'.format('<li>'.join([''] + out_rows))

#def get_addresses(dep):
#    file = 'C:\\deconMAP\\adresses_treated\\ban-' + str(dep) + '.csv.gz'
#    return pd.read_csv(file, compression='gzip', sep=',', low_memory=False)

def handle_interaction(**kwargs):
    if kwargs['type'] == 'click':
        found_address = geolocator.reverse(str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]), language='fr')
        if found_address :
            msg = '( %s ) Selected coordinates: lat %s , lon %s (identifié à %s metres du click %s)\n' % (datetime.now(), found_address.latitude, found_address.longitude, 
                                                                                                          geodesic((found_address.latitude, found_address.longitude),kwargs['coordinates']).m, kwargs['coordinates'])
            if found_address.address:
                if found_address.address.split(', ')[-1] in list(pd.read_csv(countries_file, sep=',', low_memory=False, header=None)[4]):
                    state_selector.value = found_address.address.split(', ')[-1]
                    if state_selector.value == 'France':
                        user_selection.value = found_address.address
                    else:
                        user_selection.value = found_address.address
                    msg += '-> %s\n' % (state_selector.value)
                    circle.location = (kwargs['coordinates'][0], kwargs['coordinates'][1])
                    circle.radius = 1
                    circle.color = "blue"
                    main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle]
                    circle_radius.location = (0, 0)
                    circle_radius.radius = 0
                else:
                    msg += '-> (ERROR) %s : pays non disponible \n' % (found_address.address.split(', ')[-1])
            else:
                msg += '-> (ERROR) aucune adresse trouvée ici \n'
        else:
            msg = '( ERROR - %s ) lat %s , lon %s : coordonnées non disponibles \n' % (datetime.now(), kwargs['coordinates'][0], kwargs['coordinates'][1])
        out.value = add_log(msg)

def on_map_selected(change):
    main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle, circle_radius]

def on_state_selected(change):
    if (geolocator.geocode(state_selector.value, language='fr')):
        center_map = geolocator.geocode(state_selector.value)
        main_map.center = (center_map.latitude, center_map.longitude)
        main_map.zoom = 5
    else:
        msg = '( ERROR - %s ) %s : pays non disponible \n' % (datetime.now(), state_selector.value)    
        out.value = add_log(msg)

def on_button_clicked(b):
    #main_map.fireEvent('click',{latlng:[x,y]})
    found_address = geolocator.geocode(user_selection.value, language='fr')
    if(found_address):
        msg = '( %s ) Result : %s (%s , %s)\n' % (datetime.now(), found_address.address, found_address.latitude, found_address.longitude)
        if found_address.address.split(', ')[-1] in list(pd.read_csv(countries_file, sep=',', low_memory=False, header=None)[4]):
            state_selector.value = found_address.address.split(', ')[-1]
            msg += '-> %s\n' % (state_selector.value)
            circle.location = (found_address.latitude, found_address.longitude)
            circle.radius = 1
            circle.color = "blue"
            main_map.layers = [basemap_to_tiles(maps[basemap_selector.value]), circle]
            circle_radius.location = (found_address.latitude, found_address.longitude)
            circle_radius.radius = 100000
            circle_radius.color = "red"
            circle_radius.fill_color = "green"
            main_map.add_layer(circle_radius)
        else:
            msg += '-> (ERROR) %s : pays non disponible \n' % (found_address.address.split(', ')[-1])
    else:
        msg = '( ERROR - %s ) %s : adresse non disponible \n' % (datetime.now(), user_selection.value)    
    out.value = add_log(msg)


basemap_selector.observe(on_map_selected, names='value')

state_selector.observe(on_state_selected, names='value')

main_map.on_interaction(handle_interaction)

user_validation.on_click(on_button_clicked)    


grid_header = GridspecLayout(1, 4, height='auto')
grid_header[0, :3] = header
grid_header[0, 3] = VBox([Label("Pays:"),state_selector])
#grid_header[0, 3] = VBox([Label("Basemap:"),basemap_selector])

grid_footer = GridspecLayout(4, 1, height='auto',width='auto')
grid_footer[0,0] = VBox([Label("Adresse choisie:"),Box([user_selection,user_validation])])
grid_footer[1:2,0] = Box()
grid_footer[3,0] = out

AppLayout(header=grid_header,
          center=main_map,
          footer=grid_footer,
          #pane_widths=[0, 1, 0],
          pane_heights=['5%', '20%', '75%'],
          #width='1000px',
          height='2000px',
          grid_gap="5px")
